In [ ]:
# advanced autcorrect and autocomplete model

In [ ]:
from datasets import load_dataset

# dataset = load_dataset("tweet_eval", "emoji")
# dataset = load_dataset("Skylion007/openwebtext",split='train[:1%]')
dataset = load_dataset("bookcorpus",split='train[:50%]')


In [ ]:
dataset

In [ ]:
ds=dataset.to_pandas()
ds=ds['text']

In [ ]:
print(ds)
print(type(ds))

In [ ]:
import nltk   
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt      
import random

In [ ]:
print(type(ds[0]))
print('number of the train dataset :',len(ds))
print('\033[92m' + ds[random.randint(0,5000)])

In [ ]:
import re                                  
import string                              
from nltk.corpus import stopwords  
from nltk.tokenize import sent_tokenize

In [ ]:
# download the stopwords from NLTK
nltk.download('stopwords')

In [ ]:
#Import the english stop words list from NLTK
stopwords= stopwords.words('english')

In [ ]:
def text_processing(text_example):
    
    # convert all letters to lower case
    example = text_example.lower()

    # Remove links
    example = re.sub(r'http\S+|www.\S+|@|️#|', '', example)

    # Remove other non-alphanumeric characters 
    example = re.sub(r'[^a-zA-Z0-9 .]', ' ', example)
    
    # Tokenize the sentence
    sentence_tokens = word_tokenize(example)
    
    sentence_tokens = [token for token in sentence_tokens if token.strip() and token not in stopwords and token !='#']
        
    sentence_tokens = [token for token in sentence_tokens if token and token != '️']
    
    sentence_tokens =['<s>']+sentence_tokens +['</s>']
    
    return sentence_tokens


# def text_processing(text_example):
#     # convert all letters to lower case
#     example = text_example.lower()

#     # Remove links
#     example = re.sub(r'http\S+|www.\S+|@|️#|', '', example)
    

#     # Split the text into sentences
#     sentences = sent_tokenize(example)
    
    
#     processed_text = []
#     for sentence in sentences:
        

    
#         # Tokenize the sentence
#         sentence_tokens = nltk.word_tokenize(sentence)
        
            
#         sentence_tokens = [token for token in sentence_tokens if token.strip() and token not in stopwords and token !='#']
        
#         sentence_tokens = [token for token in sentence_tokens if token and token != '️']
        
#         sentence_tokens =['<s>']+sentence_tokens +['</s>']

#         processed_text.extend(sentence_tokens)
    
#     return processed_text

print('original text: ',ds[0])
print('processing text : ',text_processing(ds[0]))
    

In [ ]:
ds=ds[:20000]

In [ ]:
ds=ds.apply(text_processing)

In [ ]:
def check_token(token, tokenized_sentences):
    for sentence in tokenized_sentences:
        if token in sentence:
            return True
    return False

token = '#'
exists = check_token(token, ds)

print(f"Does the token '{token}' exist in the tokenized sentences? {exists}")


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(ds)
total_words = len(tokenizer.word_index)+1

In [ ]:
from collections import Counter
all_words = [word for sentence in ds for word in sentence]
print(len(Counter(all_words)))

In [ ]:
print(len(tokenizer.word_index))
print(total_words)

In [ ]:
def generate_sequences(sentences):
    inputs = []
    targets = []
    
    for sentence in sentences:
        for i in range(1, len(sentence)):
            inputs.append(sentence[:i])
            targets.append(sentence[i])
            
    return inputs, targets

inputs, targets = generate_sequences(ds)
print("Inputs:", inputs[:2])
print("Targets:", targets[:2])


In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
def plot_sequence_lengths(sequences):
    # Calculate the lengths of the sequences
    lengths = [len(seq) for seq in sequences]

    # Count the frequencies of each length
    counter = Counter(lengths)

    # Separate the lengths and their frequencies
    lengths = list(counter.keys())
    frequencies = list(counter.values())

    # Plot the frequencies of each length
    plt.bar(lengths, frequencies)
    plt.xlabel('Sequence Length')
    plt.ylabel('Frequency')
    plt.title('Frequencies of Sequence Lengths')
    plt.show()



# Plot the frequencies of the sequence lengths
plot_sequence_lengths(inputs)


In [ ]:
import numpy as np

def find_optimal_maxlen(sequences):
    # Calculate the lengths of the sequences
    lengths = [len(seq) for seq in sequences]

    # Calculate the mean and standard deviation of the lengths
    mean = np.mean(lengths)
    std = np.std(lengths)

    # Set the optimal max length to be the mean plus one standard deviation
    optimal_maxlen = int(mean + std)

    return optimal_maxlen


# Find the optimal max length for padding
optimal_maxlen = find_optimal_maxlen(inputs)

print(f"The optimal max length for padding is {optimal_maxlen}.")


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

def generate_tokenized_sequences(sentences,total_words):
#     # Initialize the tokenizer
#     tokenizer = Tokenizer()
    
#     # Flatten the list of sentences and fit the tokenizer
#     all_words = [word for sentence in sentences for word in sentence]
#     tokenizer.fit_on_texts(all_words)
    
    sequences = []
    targets = []
    
    for sentence in sentences:
        for i in range(1, len(sentence)):
            # Convert the sentence up to i+1 to a sequence
            sequence = tokenizer.texts_to_sequences([sentence[:i+1]])[0]
            sequences.append(sequence)
    
    # Pad the sequences
    sequences = pad_sequences(sequences)
    
    # Split the padded sequences into inputs and targets
    inputs = sequences[:, :-1]
    targets = sequences[:, -1]
    targets = to_categorical(targets, num_classes=total_words)

    
    return inputs, targets #,tokenizer

inputs, targets = generate_tokenized_sequences(ds,total_words)

print("Inputs:", inputs)
print("Targets:", targets)


In [ ]:
print(targets[:5])

In [ ]:
def check_sequences_length(sequences):
    # Get the length of the first sequence
    first_sequence_length = len(sequences[0])

    # Check if all sequences have the same length
    return all(len(sequence) == first_sequence_length for sequence in sequences)

print(check_sequences_length(inputs))  # Outputs: True


In [ ]:
vocab_size =len(tokenizer.word_index)+1
maxlen=len(inputs[0])
maxlen

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

# vocab_size, embedding_dim = embedding_matrix.shape
# print(vocab_size, embedding_dim)
model = Sequential([
#     Embedding(input_dim=vocab_size, output_dim=embedding_dim, trainable=False),
    Embedding(total_words, 300),
    Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2)),
    Bidirectional(LSTM(units=128, dropout=0.2)),
    Dense(units=128, activation='relu'),
#     Dropout(0.5),
    Dense(units=vocab_size, activation='softmax')
])

# model.build(input_shape=(None, maxlen))

# # Now set the weights of the Embedding layer
# model.layers[0].set_weights([embedding_matrix])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size = 300
history = model.fit(inputs, targets,batch_size=batch_size,epochs=10)

In [ ]:
def data_generator(inputs, targets, batch_size):
    num_samples = len(inputs)
    while True:  # Loop forever so the generator never terminates
        for i in range(0, num_samples, batch_size):
            batch_inputs = inputs[i:i+batch_size]
            batch_targets = targets[i:i+batch_size]
            yield batch_inputs, batch_targets

batch_size = 2860
train_generator = data_generator(inputs, targets, batch_size)

history = model.fit(train_generator, steps_per_epoch=len(inputs)//batch_size, epochs=10)

In [ ]:
import matplotlib.pyplot as plt



# Plotting the loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()

# Plotting the accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
